In [30]:
import subprocess
import os
import pandas as pd
import numpy as np
import pylab as plt
import re

In [4]:
df_images = pd.read_csv('ProstateX-TrainingLesionInformationv2/ProstateX-TrainingLesionInformationv2/ProstateX-Images-Train.csv')
df_findings = pd.read_csv('ProstateX-TrainingLesionInformationv2/ProstateX-TrainingLesionInformationv2/ProstateX-Findings-Train.csv')

In [14]:
df_images[df_images['DCMSerDescr'] == 't2_tse_tra']


,ProxID,Name,fid,pos,WorldMatrix,ijk,TopLevel,SpacingBetweenSlices,VoxelSpacing,Dim,DCMSerDescr,DCMSerNum
7,ProstateX-0000,t2_tse_tra0,1,25.7457 31.8707 -38.511,"0.5,1.00168e-010,0.00377059,-57.9373,-0.000199...",167 224 9,1.0,3.0,"0.5,0.5,3",384x384x19x1,t2_tse_tra,4
19,ProstateX-0001,t2_tse_tra0,1,-40.5367071921656 29.320722668457 -16.70766907...,"0.5,1.02552e-010,-4.64581e-017,-119.205,-1.019...",157 186 10,1.0,3.0,"0.5,0.5,3",384x384x19x1,t2_tse_tra,6
20,ProstateX-0001,t2_tse_tra0,1,-40.5367071921656 29.320722668457 -16.70766907...,"0.5,1.02552e-010,-4.64581e-017,-119.205,-1.019...",157 186 10,1.0,3.0,"0.5,0.5,3",384x384x19x1,t2_tse_tra,10
40,ProstateX-0002,t2_tse_tra0,2,-2.058 38.6752 -34.6104,"0.5,1.02552e-010,7.98512e-017,-103.784,-9.9634...",203 216 10,1.0,3.0,"0.5,0.5,3",384x384x19x1,t2_tse_tra,4
41,ProstateX-0002,t2_tse_tra0,1,-27.0102 41.5467 -26.0469,"0.5,1.02552e-010,7.98512e-017,-103.784,-9.9634...",154 217 12,1.0,3.0,"0.5,0.5,3",384x384x19x1,t2_tse_tra,4
60,ProstateX-0003,t2_tse_tra0,1,22.1495 31.2717 -2.45933,"0.5,1.02552e-010,3.21757e-017,-98.0005,-8.5615...",240 205 12,1.0,3.0,"0.5,0.5,3",384x384x21x1,t2_tse_tra,3
61,ProstateX-0003,t2_tse_tra0,2,-21.2871 19.3995 19.7429,"0.5,1.02552e-010,3.21757e-017,-98.0005,-8.5615...",153 161 17,1.0,3.0,"0.5,0.5,3",384x384x21x1,t2_tse_tra,3
71,ProstateX-0004,t2_tse_tra0,1,-7.69665 3.64226 23.1659,"0.5,1.02552e-010,6.33633e-017,-118.163,-9.9505...",221 168 7,1.0,4.5,"0.5,0.5,4.5",384x384x19x1,t2_tse_tra,5
94,ProstateX-0005,t2_tse_tra0,1,-22.0892639160156 25.4668045043945 22.87915420...,"0.5,1.02552e-010,1.10673e-016,-117.278,-1.0216...",190 185 10,1.0,3.0,"0.5,0.5,3",384x384x19x1,t2_tse_tra,4
95,ProstateX-0005,t2_tse_tra0,0,-14.5174331665039 49.4428329467773 20.78152465...,"0.5,1.02552e-010,1.10673e-016,-117.278,-1.0216...",206 233 10,1.0,3.0,"0.5,0.5,3",384x384x19x1,t2_tse_tra,4


In [15]:
exe = "../bin/Debug/registration.exe"
images_source = "C:\\FRANTISEK\\SKOLA FIIT\\bakalarka\\DATA\\PROSTATEx\\"
filename1 = "C:\\FRANTISEK\\SKOLA FIIT\\bakalarka\\DATA\\PROSTATEx\\ProstateX-0000\\07-07-2011-MR prostaat kanker detectie WDSmc MCAPRODETW-05711\\4-t2tsetra-00702"
filename2 = "C:\\FRANTISEK\\SKOLA FIIT\\bakalarka\\DATA\\PROSTATEx\\ProstateX-0000\\07-07-2011-MR prostaat kanker detectie WDSmc MCAPRODETW-05711\\skuska2.dcm"
i = "25.7457"
j = "31.8707"
k = "8"
deg = "20"

args = [exe,filename1,filename2,i,j,k,deg]


        

In [16]:
args


['../bin/Debug/registration.exe',
 'C:\\FRANTISEK\\SKOLA FIIT\\bakalarka\\DATA\\PROSTATEx\\ProstateX-0000\\07-07-2011-MR prostaat kanker detectie WDSmc MCAPRODETW-05711\\4-t2tsetra-00702',
 'C:\\FRANTISEK\\SKOLA FIIT\\bakalarka\\DATA\\PROSTATEx\\ProstateX-0000\\07-07-2011-MR prostaat kanker detectie WDSmc MCAPRODETW-05711\\skuska2.dcm',
 '25.7457',
 '31.8707',
 '8',
 '20']

In [17]:
p = subprocess.Popen(args, stdin=subprocess.PIPE, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

In [18]:
(out, err) = p.communicate()

print(out)

b'\r\nThe directory: \r\n\r\nC:\\FRANTISEK\\SKOLA FIIT\\bakalarka\\DATA\\PROSTATEx\\ProstateX-0000\\07-07-2011-MR prostaat kanker detectie WDSmc MCAPRODETW-05711\\4-t2tsetra-00702\r\n\r\nContains the following DICOM Series: \r\n\r\n1.3.6.1.4.1.14519.5.2.1.7311.5101.160028252338004527274326500702.4tse2d125338438420110707\r\n\r\n\r\nNow reading series: \r\n\r\n1.3.6.1.4.1.14519.5.2.1.7311.5101.160028252338004527274326500702.4tse2d125338438420110707\r\n\r\n\r\nxindex = 167\r\nYindex = 224\r\nzIndex = 8\r\nimageCenterX = 384\r\nimageCenterY = 384\r\nimageCenterX = 25.7457\r\nimageCenterY = 31.8707\r\nCenterX = 166\r\nCenterY = 228\r\nWriting the image as \r\n\r\nC:\\FRANTISEK\\SKOLA FIIT\\bakalarka\\DATA\\PROSTATEx\\ProstateX-0000\\07-07-2011-MR prostaat kanker detectie WDSmc MCAPRODETW-05711\\skuska2.dcm\r\n\r\nWriting the image as \r\n\r\nC:\\FRANTISEK\\SKOLA FIIT\\bakalarka\\DATA\\PROSTATEx\\ProstateX-0000\\07-07-2011-MR prostaat kanker detectie WDSmc MCAPRODETW-05711\\skuska2.dcm\r\n\r

In [27]:
dir1 = "C:\\FRANTISEK\\SKOLA FIIT\\bakalarka\\DATA\\PROSTATEx\\ProstateX-0000\\"
dirs = os.listdir(dir1)

In [28]:
a = os.path.join(dir1, dirs[0])

In [53]:

dirs2 = os.listdir(a)
regex = r"^" + str(4)
next(filter(lambda x: x.startswith( str(4) ),dirs2))


'4-t2tsetra-00702'

In [35]:
dirs2

['3-t2tsesag-87368',
 '4-t2tsetra-00702',
 '5-t2tsecor-03471',
 '6-ep2ddifftraDYNDIST-69284',
 '7-ep2ddifftraDYNDISTADC-48780',
 '8-ep2ddifftraDYNDISTCALCBVAL-83202',
 'man.dcm',
 'man2.dcm',
 'moj.dcm',
 'neve',
 'skuska.dcm',
 'skuska2.dcm',
 'tvoj.dcm']

In [76]:
T2W = df_images[df_images['DCMSerDescr'] == 't2_tse_tra']


In [69]:
root_path = "C:\\FRANTISEK\\SKOLA FIIT\\bakalarka\\DATA\\PROSTATEx"
for index , cas in T2W.iterrows():
    
    DICOMpath = os.path.join(root_path,str(cas['ProxID']))
    dirs1 = os.listdir(DICOMpath)
    
    DICOMpath = os.path.join(DICOMpath,dirs1[0])
    dirs2 = os.listdir(DICOMpath)
    r1 = next(filter(lambda x: x.startswith( str(cas['DCMSerNum']) ),dirs2))   
    DICOMpath = os.path.join(DICOMpath,r1)
    
    T2W.loc[index,'path'] = DICOMpath
    

C:\Anaconda3\lib\site-packages\pandas\core\indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
C:\Anaconda3\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [75]:
T2W.loc[7,'path']


'C:\\FRANTISEK\\SKOLA FIIT\\bakalarka\\DATA\\PROSTATEx\\ProstateX-0000\\07-07-2011-MR prostaat kanker detectie WDSmc MCAPRODETW-05711\\4-t2tsetra-00702'